Clickable Histogram (ClickHist) + Atmospheric Data Input =
Clickable Histogram of Atmospheric Data (CHAD)

Author: Matthew Niznik (matthew.niznik9@gmail.com)<br>
Post-Doctoral Associate, RSMAS, University of Miami

For more information, see:<br>
https://sites.google.com/site/matthewjniznik/research/<br>
https://sites.google.com/site/matthewjniznik/research/chad

(Note: iPython Notebook needs a few tweaks to work most seemlessly with ClickHist and ClickHistDo instances - those will be pointed out below as they come up.)

In [1]:
#----- User Changeable Parameters (and appropriate libraries) -----

#----- Figure Size and Resolution -----
#Set the figure x by y resolution, DPI, and the max number of points to appear in a given bin
#(Plotting time as well as finding an individual event prohibitive for very large maxPlottedInBin values)
#(These are OPTIONAL inputs to ClickHist: figX=?, figY=?, figDPI=?, maxPlottedInBin=?)
figureXSize = 800
figureYSize = 800
figDPI = 150
maxPlottedInBin_UD = 1000

#----- Formatting for Output -----
#Basic Help: The number after the decimal point sets the number of decimal points shown in output
#For more on Python string formatting, see:
#(https://mkaz.github.io/2012/10/10/python-string-format/)
#These are OPTIONAL inputs to ClickHist: xFmtStr=?,yFmtStr=?)
var1FmtStr = "{:2.0f}"
var2FmtStr = "{:4.0f}"

#----- Set the URL/Filepath for load files as well as the variable names to load -----
#Note: Loading bin edges is no longer necessary - they could also be manually specified as a numpy array
#and passed to ClickHist. This is a relic of when ClickHist needed a histogram passed to it.

#urlToLoadHist = '/path/to/your/directory'
urlToLoad = 'https://weather.rsmas.miami.edu/repository/opendap/synth:4fd8f0b4-7cc2-411a-9b61-2ef0c24c1637:L0hWX3I5MHg0NV8zXzBFNU5fSE1WX0tFRE9ULm5jNA==/entry.das'

#----- Variable names in input file(s) for bin edges and values -----
var1EdgeName = 'HMVBINEDGES'
var2EdgeName = 'KEDOTBINEDGES'
var1ValueName = 'HMV'
var2ValueName = 'KEDOT'

#----- Variable names in input file(s) for data needed for ClickHistDo_IDV -----
lonValueName = 'lon'
latValueName = 'lat'
timeValueName = 'time'

#----- Variable Names and Units -----
#These are optional descriptive inputs to both ClickHist and (some) to ClickHistDo
#so that the ClickHist and the output bundle are labeled properly
var1Name = 'HMV'
var2Name = 'KEDot'
var1Units = 'm2 s-2'
var2Units = 'm3 s-3'
metadata_UD = 'HMV vs KEDot: 0-36 E, 10-30 N'

#----- Unit correction options -----
#If the units in the input file are not what is desired, they can be corrected during the load with
#these multipliers.
var1ValueMult = 1.
var2ValueMult = 1.

In [2]:
#----- Setting the GUI -----
#ClickHist is currently optimized for tk
#For more options see section "%matplotlib" at
#https://ipython.org/ipython-doc/3/interactive/magics.html

#----- Matplotlib for graphics, set tk too -----
%matplotlib tk
import matplotlib

#----- Modules for fixing the buffer in cell 3 -----
from IPython.display import clear_output
import sys

#----- numpy to create the sample input arrays -----
import numpy as np

#----- supress warnings from log10(0) -> handled in code -----
#----- Would rather not supress ALL warnings right now - fixed at a later date? -----
#import warnings
#warnings.filterwarnings('ignore')

#----- And obviously import ClickHist and ClickHistDo! -----
import ClickHist_IDV as ClickHist
import ClickHistDo_IDV as ClickHistDo

#----- User-specified imports -----
#Import netCDF4 to load the netCDF input file
import netCDF4

In [3]:
#----- Fixing the output so it isn't buffered -----
#See: http://stackoverflow.com/questions/29772158/make-ipython-notebook-print-in-real-time
#Note: Nothing to edit in this cell!

oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

In [4]:
#----- Manual Bin Definition -----
#Later call to create ClickHist uses the below variable names
#You should probably leave the names alone
#var1Edges = np.arange(0,1+0.01,0.1)
#var2Edges = np.arange(0,1+0.01,0.1)

#----- Load the Data -----
cdfIn = netCDF4.Dataset(urlToLoad,'r')

#----- Bin Edge and Value Data -----
#Later call to create ClickHist uses the below variable names
#You should probably leave the names alone
var1Edges = cdfIn.variables[var1EdgeName][:]
var2Edges = cdfIn.variables[var2EdgeName][:]
var1Values = cdfIn.variables[var1ValueName][:]*var1ValueMult
var2Values = cdfIn.variables[var2ValueName][:]*var2ValueMult

#----- Data for ClickHistDo_IDV -----
lonValues = cdfIn.variables[lonValueName][:]
latValues = cdfIn.variables[latValueName][:]
timeValues = cdfIn.variables[timeValueName][:]
cdfIn.close()

In [5]:
#----- Create ClickHist using a proper call -----
#If you only changed variable values in cells 1 and 4 above,
#ClickHist is ready to go!

#This call is necessary to create the output console for ClickHist
%qtconsole

#----- Create a ClickHistDo instance -----
ClickHistDo1 = ClickHistDo.ClickHistDo(lonValues,latValues,timeValues,
                                      xVarName=var1ValueName,yVarName=var2ValueName)
#----- Create a ClickHist instance -----
ClickHist1 = ClickHist.ClickHist(var1Edges,var2Edges,var1Values,var2Values,
                                xVarName=var1Name,yVarName=var2Name,
                                xUnits=var1Units,yUnits=var2Units,
                                xFmtStr=var1FmtStr,yFmtStr=var2FmtStr,
                                maxPlottedInBin=maxPlottedInBin_UD,
                                metadata=metadata_UD)
#----- Set ClickHistDo1 to be the official "action" for ClickHist -----
ClickHist1.setDo(ClickHistDo1)

#----- Show the ClickHist -----
ClickHist1.showPlot()

Saving IDV bundle...
2005-06-08 04:30:00
Saved!
